In [2]:
import os

In [4]:
os.chdir("../")
%pwd

'/Users/harshithreddy/Desktop/AdvanceMachineLearning/chickenDiseaseClassification'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url, 
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion: 
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-08 08:12:44,904: INFO: common: yaml file: config/config.yaml loaded sucessfully]
[2025-06-08 08:12:44,918: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-06-08 08:12:44,926: INFO: common: created directory at: artifacts]
[2025-06-08 08:12:44,929: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-08 08:12:46,352: INFO: 2419591356: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11621297
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c59eb086f697a4734c2bda47f04f88206df50dfc5d0dcc683113e225dfde848"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A9D4:356982:83D6D1:A4881E:68458C4C
Accept-Ranges: bytes
Date: Sun, 08 Jun 2025 13:12:46 GMT
Via: 1.1 varnish
X-Served-By: cache-mci680044-MCI
X-Cache: 